<h1>Hierarchical Clusternig of 30 min data blocks</h1>
<p>In order to attempt to make better clusters we combined 30 minutes worth of data. For example a dishwasher wil use both water and electricity but not necessarily at exactly the same moment.</p>

In [0]:
%%bash
pip install -U scikit-learn

In [0]:
import numpy as np

import os
import inspect
import sys
import pandas as pd
import charts
import time

from opengrid.library import houseprint

from sklearn.cluster import MeanShift

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [0]:
hp = houseprint.Houseprint()

In [0]:
device = hp.find_device('FL03001441')
device.get_sensors()

In [0]:
#Get Electricity data
head = pd.Timestamp('20151114')
tail = pd.Timestamp('20151118')
electr = hp.find_sensor('81fb589c911e808daa96a2064bebed41')
df_el = electr.get_data(head,tail,diff=True, unit='W')
df_el = df_el.dropna()

In [0]:
#Get Gas data
gas = hp.find_sensor('212ce724e124fbde0fb649396375d099')
df_ga = gas.get_data(head,tail,diff=True)
df_ga = df_ga.dropna()

In [0]:
#Get Water data
water = hp.find_sensor('c57f45b21e2ec21ab427f172478e1ca4')
df_wa = water.get_data(head,tail,diff=True)
df_wa = df_wa.dropna()

<h3>Gas vs Water</h3>

In [0]:
#Concatinate the two datasets and drop lines with zero values
result = pd.concat([df_wa, df_ga], axis=1).dropna()
result.columns = ['water', 'gas']

<p>The duration of a block and the overlap of two blocks can be altered to have better results</p>

In [0]:
#Create new dataframe for cumulative consumption
df_gawa = pd.DataFrame({'gas' : [], 'water':[]})

df_loop = result
while (df_loop.size > 0):
    #Get timestamp of first row and take all rows between this time and 30 minutes later
    ts_start = df_loop.index[0]
    ts_end = (ts_start + 29)
    df_temporary = df_loop.ix[ts_start:ts_end]
    
    #Remove these first 30 minutes from the list
    df_loop = df_loop.ix[30:]
    
    #Sum these values and store them in the newely created dataframe
    df_gawa = df_gawa.append(pd.DataFrame([[df_temporary.sum()['gas'], df_temporary.sum()['water']]],index = [ts_start] , columns=df_gawa.columns))

    

In [0]:
df_gawa = df_gawa[~((df_gawa-df_gawa.mean()).abs()>3*df_gawa.std())]
df_gawa = df_gawa.dropna()

df_gawa['gas'] = df_gawa['gas']/df_gawa['gas'].max()
df_gawa['water'] = df_gawa['water']/df_gawa['water'].max()

In [0]:
#convert pandas to np array
resultArray = df_gawa.as_matrix()

In [0]:
#Run clustering Algorithm
ms = MeanShift()
ms.fit(resultArray)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_cluster = len(np.unique(labels))
colors = 10*['g.', 'r.', 'c.', 'b.', 'k.', 'y.', 'm.']

In [0]:
#Plot clusters
for i in range(len(resultArray)):
    plt.plot(resultArray[i][0], resultArray[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1], marker = "x", s = 150, linewidths = 5, zorder = 10)
plt.xlabel('Gas')
plt.ylabel('Water')
plt.show()

<h3>Electricity vs Water</h3>

In [0]:
#Concatinate the two datasets and drop lines with zero values
result = pd.concat([df_wa, df_el], axis=1).dropna()
result.columns = ['water', 'electricity']

<p>The duration of a block and the overlap of two blocks can be altered to have better results</p>

In [0]:
#Create new dataframe for cumulative consumption
df_elwa = pd.DataFrame({'electricity' : [], 'water':[]})

df_loop = result
while (df_loop.size > 0):
    #Get timestamp of first row and take all rows between this time and 30 minutes later
    ts_start = df_loop.index[0]
    ts_end = (ts_start + 29)
    df_temporary = df_loop.ix[ts_start:ts_end]
    
    #Remove these first 30 minutes from the list
    df_loop = df_loop.ix[30:]
    
    #Sum these values and store them in the newely created dataframe
    df_elwa = df_elwa.append(pd.DataFrame([[df_temporary.sum()['electricity'], df_temporary.sum()['water']]],index = [ts_start] , columns=df_elwa.columns))

    

In [0]:
#df_elwa = df_elwa[~((df_elwa-df_elwa.mean()).abs()>3*df_elwa.std())]
#df_elwa = df_elwa.dropna()

df_elwa['electricity'] = df_elwa['electricity']/df_elwa['electricity'].max()
df_elwa['water'] = df_elwa['water']/df_elwa['water'].max()

In [0]:
#convert pandas to np array
resultArray = df_elwa.as_matrix()

In [0]:
#Run clustering Algorithm
ms = MeanShift()
ms.fit(resultArray)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_cluster = len(np.unique(labels))
colors = 10*['g.', 'r.', 'c.', 'b.', 'k.', 'y.', 'm.']

In [0]:
#Plot clusters
for i in range(len(resultArray)):
    plt.plot(resultArray[i][0], resultArray[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1], marker = "x", s = 150, linewidths = 5, zorder = 10)
plt.xlabel('electricity')
plt.ylabel('Water')
plt.show()

<h3>Electricity vs Gas</h3>

In [0]:
#Concatinate the two datasets and drop lines with zero values
result = pd.concat([df_ga, df_el], axis=1).dropna()
result.columns = ['gas', 'electricity']

<p>The duration of a block and the overlap of two blocks can be altered to have better results</p>

In [0]:
#Create new dataframe for cumulative consumption
df_elga = pd.DataFrame({'electricity' : [], 'gas':[]})

df_loop = result
while (df_loop.size > 0):
    #Get timestamp of first row and take all rows between this time and 30 minutes later
    ts_start = df_loop.index[0]
    ts_end = (ts_start + 29)
    df_temporary = df_loop.ix[ts_start:ts_end]
    
    #Remove these first 30 minutes from the list
    df_loop = df_loop.ix[30:]
    
    #Sum these values and store them in the newely created dataframe
    df_elga = df_elga.append(pd.DataFrame([[df_temporary.sum()['electricity'], df_temporary.sum()['gas']]],index = [ts_start] , columns=df_elga.columns))

    

In [0]:
#df_elwa = df_elwa[~((df_elwa-df_elwa.mean()).abs()>3*df_elwa.std())]
#df_elwa = df_elwa.dropna()

df_elga['electricity'] = df_elga['electricity']/df_elga['electricity'].max()
df_elga['gas'] = df_elga['gas']/df_elga['gas'].max()

In [0]:
#convert pandas to np array
resultArray = df_elga.as_matrix()

In [0]:
#Run clustering Algorithm
ms = MeanShift()
ms.fit(resultArray)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_cluster = len(np.unique(labels))
colors = 10*['g.', 'r.', 'c.', 'b.', 'k.', 'y.', 'm.']

In [0]:
#Plot clusters
for i in range(len(resultArray)):
    plt.plot(resultArray[i][0], resultArray[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1], marker = "x", s = 150, linewidths = 5, zorder = 10)
plt.xlabel('electricity')
plt.ylabel('gas')
plt.show()